<a href="https://colab.research.google.com/github/dainfinity/miQ-z/blob/main/QMusic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ★Quantum Music by miQ'z★
(ひとまず定式化の実装だけなので、3/11発表時での方針、コスト関数用いる)

# 1.概要
- 気分に合わせた伴奏のコード進行を選択、量子アニーリングの最適化により、伴奏に合った主旋律を生成する。

- 生成された音楽を聴いてもらい、フィードバックをもらう。それに応じてQUBOをPersonalizeしていく。

参考: https://qard.is.tohoku.ac.jp/T-Wave/?p=3093



# 2.定式化
どの音が、どのタイミングでなるかという変数を用意し、その組み合わせを最適化していく。


**$x_{ij}$ : $i$番目に$j$音が鳴る。**

$i\in N = \{0,1,2,3,...\}$

$j\in P = \{C4,D4,E4,F4,G4,A4,B4\}$ など(もっと用意してもよい)

例えば、4分の4拍子を仮定して、$n = \mathrm{len}(N) = 8$とすると、これは2小節分生成するという事になる。

## 2.1 制約: 1音節には1音のみ

1音節には1音のみになってほしいので、

\begin{equation}
\displaystyle \sum_j x_{ij} = 1
\end{equation}

となる必要がある。この制約は非常に強くするべき。

## 2.2 制約: 同じ音の連続を防ぐ

連続して何度も同じ音が鳴ってしまってはつまらないので、同じ音は連続2回まで許すことにする。ある音jが2回より多くならないということは

\begin{equation}
\displaystyle \sum_i x_{i,j} + x_{i+1,j} + x_{i+2,j} \leq 2 
\end{equation}

で表すことができる。

## 2.3 制約: 単体でなると嫌な音
(調査不足なので、一旦飛ばす)


## 2.4 制約: 伴奏のコードに合わせた制約
伴奏のコードが「C」だった場合、主旋律では「ド、ミ、ソ」(C4,E4,G4)のみを使うのが望ましい。
一通り楽譜を見た時に、「音符の個数 = ドミソの合計」になっていてほしいので、

\begin{equation}
\displaystyle \sum_i x_{i,C4} + x_{i,E4} + x_{i,G4} = n
\end{equation}

これはもっと一般化する必要があるが、ここでは一旦伴奏のコードが「C」の場合だけ考えて実装する。

## 2.5 ブラックボックス最適化
ユーザの好みに合わせて、QUBOを更新していく。

今回はまだフィードバックもくそもないので、適当に。

------実装-----

In [1]:
pip install dwave-ocean-sdk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.8/74.8 KB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.5/245.5 KB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.4/111.4 KB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 59.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 59.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 KB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 KB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 KB 373.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [2]:
token = "みんなのtokenを入れる" 

In [3]:
import numpy as np
from dwave.system import DWaveSampler, EmbeddingComposite
import dimod
import neal

In [8]:
P = ["C4","D4","E4","F4","G4","A4","B4"] #ド、レ、ミ、ファ、ソ、ラ、シ
n = 8 #4/4で2小節分

In [33]:
# 変数x_{ij}を定義
x = []
for i in range(n):
    for j in P:
        x.append(dimod.Binary(f'x_{i,j}'))

In [34]:
x

[BinaryQuadraticModel({"x_(0, 'C4')": 1.0}, {}, 0.0, 'BINARY'),
 BinaryQuadraticModel({"x_(0, 'D4')": 1.0}, {}, 0.0, 'BINARY'),
 BinaryQuadraticModel({"x_(0, 'E4')": 1.0}, {}, 0.0, 'BINARY'),
 BinaryQuadraticModel({"x_(0, 'F4')": 1.0}, {}, 0.0, 'BINARY'),
 BinaryQuadraticModel({"x_(0, 'G4')": 1.0}, {}, 0.0, 'BINARY'),
 BinaryQuadraticModel({"x_(0, 'A4')": 1.0}, {}, 0.0, 'BINARY'),
 BinaryQuadraticModel({"x_(0, 'B4')": 1.0}, {}, 0.0, 'BINARY'),
 BinaryQuadraticModel({"x_(1, 'C4')": 1.0}, {}, 0.0, 'BINARY'),
 BinaryQuadraticModel({"x_(1, 'D4')": 1.0}, {}, 0.0, 'BINARY'),
 BinaryQuadraticModel({"x_(1, 'E4')": 1.0}, {}, 0.0, 'BINARY'),
 BinaryQuadraticModel({"x_(1, 'F4')": 1.0}, {}, 0.0, 'BINARY'),
 BinaryQuadraticModel({"x_(1, 'G4')": 1.0}, {}, 0.0, 'BINARY'),
 BinaryQuadraticModel({"x_(1, 'A4')": 1.0}, {}, 0.0, 'BINARY'),
 BinaryQuadraticModel({"x_(1, 'B4')": 1.0}, {}, 0.0, 'BINARY'),
 BinaryQuadraticModel({"x_(2, 'C4')": 1.0}, {}, 0.0, 'BINARY'),
 BinaryQuadraticModel({"x_(2, 'D4')": 1.

In [67]:
# cqmに問題を乗せるおまじない
cqm = dimod.ConstrainedQuadraticModel()

In [68]:
# ユーザからのフィードバックで逐次更新されていくはずのQUBO行列をゼロで初期化
p = len(P) #音のバリエーションの個数
QUBO_tensor = np.zeros([n,p,n,p]) #Q_{ijkl}という4次元配列
QUBO_mat = QUBO_tensor.reshape(n*p, n*p) #Q_{ijkl}を行列に整形

In [70]:
# コスト関数を定義(制約でない部分)
cqm.set_objective(sum(sum(QUBO_mat[ij][kl]*x[ij]*x[kl] for kl in range(n*p))for ij in range(n*p)))

In [71]:
# 制約 1音節に1音
for i in range(n):
    cqm.add_constraint(sum(x[i*p+j] for j in range(p)) == 1,label=f'onehot_{i}')

In [72]:
# 制約 同じ音が2回より多く連続することを防ぐ
for j in range(p):
    cqm.add_constraint(sum(x[i*p+j] for i in range(n)) <= 2, label=f'NoContinue_{j}')

In [73]:
# 制約 伴奏にコードに合わせる(実装がよくわからない)

In [74]:
bqm, invert = dimod.cqm_to_bqm(cqm,lagrange_multiplier=5)

In [75]:
# nealで検証
sampler_sim = neal.SimulatedAnnealingSampler()

sampleset = sampler_sim.sample(bqm, num_reads=100)

invert(sampleset.first.sample)

{"x_(0, 'C4')": 0,
 "x_(0, 'D4')": 0,
 "x_(0, 'E4')": 0,
 "x_(0, 'F4')": 0,
 "x_(0, 'G4')": 0,
 "x_(0, 'A4')": 1,
 "x_(0, 'B4')": 0,
 "x_(1, 'C4')": 0,
 "x_(1, 'D4')": 0,
 "x_(1, 'E4')": 0,
 "x_(1, 'F4')": 1,
 "x_(1, 'G4')": 0,
 "x_(1, 'A4')": 0,
 "x_(1, 'B4')": 0,
 "x_(2, 'C4')": 0,
 "x_(2, 'D4')": 0,
 "x_(2, 'E4')": 0,
 "x_(2, 'F4')": 0,
 "x_(2, 'G4')": 1,
 "x_(2, 'A4')": 0,
 "x_(2, 'B4')": 0,
 "x_(3, 'C4')": 1,
 "x_(3, 'D4')": 0,
 "x_(3, 'E4')": 0,
 "x_(3, 'F4')": 0,
 "x_(3, 'G4')": 0,
 "x_(3, 'A4')": 0,
 "x_(3, 'B4')": 0,
 "x_(4, 'C4')": 0,
 "x_(4, 'D4')": 1,
 "x_(4, 'E4')": 0,
 "x_(4, 'F4')": 0,
 "x_(4, 'G4')": 0,
 "x_(4, 'A4')": 0,
 "x_(4, 'B4')": 0,
 "x_(5, 'C4')": 0,
 "x_(5, 'D4')": 0,
 "x_(5, 'E4')": 0,
 "x_(5, 'F4')": 0,
 "x_(5, 'G4')": 0,
 "x_(5, 'A4')": 1,
 "x_(5, 'B4')": 0,
 "x_(6, 'C4')": 0,
 "x_(6, 'D4')": 0,
 "x_(6, 'E4')": 1,
 "x_(6, 'F4')": 0,
 "x_(6, 'G4')": 0,
 "x_(6, 'A4')": 0,
 "x_(6, 'B4')": 0,
 "x_(7, 'C4')": 0,
 "x_(7, 'D4')": 0,
 "x_(7, 'E4')": 0,
 "x_(7, 'F4'

In [87]:
# 実行結果から、何番目に何が鳴るかを取り出す
best_dict = invert(sampleset.first.sample)
best_list = []
for ij in best_dict.keys():
    if best_dict[ij] == 1:
        best_list.append(ij)    

best_list

["x_(0, 'A4')",
 "x_(1, 'F4')",
 "x_(2, 'G4')",
 "x_(3, 'C4')",
 "x_(4, 'D4')",
 "x_(5, 'A4')",
 "x_(6, 'E4')",
 "x_(7, 'G4')"]

これは、
「ラ、ファ、ソ、ド」「レ、ラ、ミ、ソ」ということである。

1音節に1音、音が2回より多く連続しないという制約は守られている。

今回は、伴奏に合わせた制約を入れていないので、今のところ満足した結果が得られている。